In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('penguins.csv')

In [6]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [4]:
! pip install sqlalchemy

In [7]:
df['island'].unique()

array(['Torgersen', 'Biscoe', 'Dream'], dtype=object)

In [10]:
from sqlalchemy import create_engine, text

In [11]:
temp_db = create_engine('sqlite:///memory:', echo=True)

In [12]:
data = df.to_sql(name='Penguins', con = temp_db)

2024-01-08 18:18:17,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-08 18:18:17,352 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2024-01-08 18:18:17,352 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-08 18:18:17,353 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2024-01-08 18:18:17,354 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-08 18:18:17,354 INFO sqlalchemy.engine.Engine ROLLBACK
2024-01-08 18:18:17,356 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-08 18:18:17,357 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2024-01-08 18:18:17,357 INFO sqlalchemy.engine.Engine [no key 0.00044s] ()
2024-01-08 18:18:17,359 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2024-01-08 18:18:17,360 INFO sqlalchemy.engine.Engine [no 

In [33]:
with temp_db.connect() as conn:
    result = conn.execute(text("""
    SELECT COUNT(*) AS female_count
    FROM Penguins
    WHERE sex = 'FEMALE';"""))

2024-01-08 18:41:51,979 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-08 18:41:51,982 INFO sqlalchemy.engine.Engine 
    SELECT COUNT(*) AS female_count
    FROM Penguins
    WHERE sex = 'FEMALE';
2024-01-08 18:41:51,983 INFO sqlalchemy.engine.Engine [generated in 0.00507s] ()
2024-01-08 18:41:51,986 INFO sqlalchemy.engine.Engine ROLLBACK


In [34]:
result.all()

[(165,)]

In [16]:
# NLP -> SQL -> CONTEXT(large string or prefix) -> needs to know table name, column name and properties


In [17]:
df.info()  # give this as context for LLM

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [41]:
def create_prefix(query):
    #query = input("What is your question about the penguins table?")
    prefix = f"""Return a SQL statement that answers the folloing query:
    {query}
    
    For a table called 'Penguins' with the following properties:
    Data columns (total 7 columns):
     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 
     
     Example Rows:
        0 	Adelie 	Torgersen 	39.1 	18.7 	181.0 	3750.0 	MALE
        1 	Adelie 	Torgersen 	39.5 	17.4 	186.0 	3800.0 	FEMALE
        2 	Adelie 	Torgersen 	40.3 	18.0 	195.0 	3250.0 	FEMALE
        3 	Adelie 	Torgersen 	NaN 	NaN 	NaN 	NaN 	NaN
        4 	Adelie 	Torgersen 	36.7 	19.3 	193.0 	3450.0 	FEMALE
    
    Only return the code, no doc strings and comments on code
    """
    return prefix

In [42]:
def user_input():
    query = input('Ask a question about the Penguins table\n')
    return create_prefix(query)

In [25]:
prefix = user_input()

Ask a question about the Penguins table
How many penguins are there in the table?


In [26]:
print(prefix)

Return a SQL statement that answers the folloing query:
    How many penguins are there in the table?
    
    For a table called 'Penguins' with the following properties:
    Data columns (total 7 columns):
     #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 
     
     Example Rows:
        0 	Adelie 	Torgersen 	39.1 	18.7 	181.0 	3750.0 	MALE
        1 	Adelie 	Torgersen 	39.5 	17.4 	186.0 	3800.0 	FEMALE
        2 	Adelie 	Torgersen 	40.3 	18.0 	195.0 	3250.0 	FEMALE
        3 	Adelie 	Torgersen 	NaN 	NaN 	NaN 	NaN 	NaN
        4 	Adelie 	Torgersen 	36.7 	19.3 	193.0 	3450.0

In [29]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/tdraly/Library/CloudStorage/Dropbox/Mac/Downloads/silent-scanner-407313-877e12e3ea43.json'

In [27]:
from vertexai.language_models import CodeGenerationModel

In [30]:
codegen_model = CodeGenerationModel.from_pretrained('code-bison')

In [43]:
result = codegen_model.predict(prefix=user_input())

Ask a question about the Penguins table
How many female penguins are there in the table?


In [46]:
result

 SELECT COUNT(*)
FROM Penguins
WHERE sex = 'FEMALE';

In [53]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql', '').replace('```','').replace('\n', ' ')

In [48]:
clean_sql(result.text)

" SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE';"

In [60]:
def nlp_assistant():
    print("Hello, I am your AI Database assistant")
    print('\n')
    prefix = user_input()
    response = codegen_model.predict(prefix=prefix)
    sql = clean_sql(response.text)
    with temp_db.connect() as conn:
        print(f'\n Ok I am running this SQL statement: \n{sql}\n\n')
        result = conn.execute(text(sql))
    print(f'\n\nHere are your results: \n{result.all()}')

In [61]:
nlp_assistant()

Hello, I am your AI Database assistant


Ask a question about the Penguins table
How many penguins are there?

 Ok I am running this SQL statement: 
 SELECT COUNT(*) AS penguin_count FROM Penguins;


2024-01-08 19:05:06,739 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-08 19:05:06,741 INFO sqlalchemy.engine.Engine  SELECT COUNT(*) AS penguin_count FROM Penguins;
2024-01-08 19:05:06,742 INFO sqlalchemy.engine.Engine [cached since 68.31s ago] ()
2024-01-08 19:05:06,745 INFO sqlalchemy.engine.Engine ROLLBACK


Here are your results: 
[(344,)]


In [56]:
nlp_assistant()

Hello, I am your AI Database assistant



Ask a question about the Penguins table
HOw many penguins are male?

 Ok I am running this SQL statement: 
 SELECT     CAST(count(*) as BIGNUMERIC)   FROM     PENGUINS   WHERE PENGUINS.SEX = 'MALE'
2024-01-08 19:02:34,978 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-08 19:02:34,979 INFO sqlalchemy.engine.Engine  SELECT     CAST(count(*) as BIGNUMERIC)   FROM     PENGUINS   WHERE PENGUINS.SEX = 'MALE'
2024-01-08 19:02:34,983 INFO sqlalchemy.engine.Engine [generated in 0.00567s] ()
2024-01-08 19:02:34,985 INFO sqlalchemy.engine.Engine ROLLBACK
Here are your results: 
[(168,)]
